In [3]:
from huggingface_hub import hf_hub_download
import os
hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
file_path = hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset", local_dir=".")
file_path

/home/skywalker/.pyenv/versions/3.10.12/envs/major/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'UCF101_subset.tar.gz'

In [4]:
import tarfile
import os
with tarfile.open("UCF101_subset.tar.gz") as t:
     t.extractall("./data")

In [5]:
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/data_handling.py
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/model_configuration.py
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/preprocessing.py

--2024-11-09 12:36:20--  https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/data_handling.py
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/olonok69/LLM_Notebooks/main/video/fine_tune_ViViT/data_handling.py [following]
--2024-11-09 12:36:21--  https://raw.githubusercontent.com/olonok69/LLM_Notebooks/main/video/fine_tune_ViViT/data_handling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2915 (2.8K) [text/plain]
Saving to: ‘data_handling.py’

data_handling.py    100%[===================>]   2.85K  --.-KB/s    in 0s      

2024-11-09 12:36:21 (51.4 MB/s) -

In [3]:
from transformers import TrainingArguments
from transformers import Trainer, TrainingArguments, AdamW
from model_configuration import *
from transformers import Trainer
from preprocessing import create_dataset
from data_handling import frames_convert_and_create_dataset_dictionary
from model_configuration import initialise_model
import wandb

In [9]:
from dotenv import load_dotenv
import os
env_path =  ".env"
load_dotenv(env_path)

True

## Base Model

In [16]:
import model_configuration
from model_configuration import compute_metrics
import cv2
import av
from data_handling import sample_frame_indices, read_video_pyav

In [18]:
container = av.open("./data/UCF101_subset/test/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c01.avi")
container.streams.video[0].frames

209

In [19]:
%pip install moviepy -q

  DEPRECATION: moviepy is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import moviepy.editor
container = av.open("./data/UCF101_subset/test/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c01.avi")
indices = sample_frame_indices(clip_len=50, frame_sample_rate=2,seg_len=container.streams.video[0].frames)
video = read_video_pyav(container=container, indices=indices)

In [22]:
indices

array([104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128,
       130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 155,
       157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181,
       183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203])

In [23]:
video.shape

(50, 224, 224, 3)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [26]:
path_files = "data/UCF101_subset"
video_dict, class_labels = frames_convert_and_create_dataset_dictionary(path_files)


Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g18_c03.avi number of Frames: 274
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g13_c05.avi number of Frames: 141
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g04_c05.avi number of Frames: 148
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g19_c06.avi number of Frames: 103
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g02_c03.avi number of Frames: 221
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g24_c05.avi number of Frames: 111
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g22_c01.avi number of Frames: 88
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g06_c05.avi number of Frames: 128
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g01_c05.avi number of Frames: 88
Processing file data/UCF101_subset/train/BenchPress/v_BenchPress_g22_c03.avi number of Frames: 74
Processing fi

In [27]:
len(video_dict)

405

In [28]:
video_dict[0].keys()

dict_keys(['video', 'labels'])

In [29]:
video_dict[0]['video'].shape

(10, 224, 224, 3)

In [30]:
video_dict[0]['labels']

'BenchPress'

In [31]:
num_frames, height, width, channels =  video_dict[0]['video'].shape
num_frames, height, width, channels 

(10, 224, 224, 3)

## Display Video Sample

In [32]:
class_labels = sorted(class_labels)
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {list(label2id.keys())}.")

Unique classes: ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress'].


In [ ]:
shuffled_dataset = create_dataset(video_dict)